<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Automatic Blog Post creator

In this project we will build a blog poster that writes a blog post given something to talk about or discuss.
Additionally, a chariming cover photo will be created!

## Imports

In [1]:
import random
import string
from pathlib import Path
import os


from git import Repo
from bs4 import BeautifulSoup as Soup


## Github Pages

1. Create a new public repository on github named **username.github.io**, where username is github username.
2. Clone the repository
3. Create a new file called **index.html**
4. Add some content to **index.html**
5. Push your changes
6. Goto https://username.github.io

## Automatic updates

Manually adding, commiting and pushing is tedious. Thus we are going to implement functionality that handles everything for us.
Before using openai to generate content for us, we evaluate our code using random strings!

In [2]:
def random_string(length):
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(length))

random_string(10)

'lvlecqvfby'

In [15]:
PATH_TO_BLOG_REPO = Path("/Users/marci/GIT/OpenAI-API-Python-Course/04-blogpost/b4shy.github.io/.git/")

def update_blog(commit_message="Updated blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()


Let's create a random text and push it to our blog!

In [ ]:
random_blogpost = random_string(1000)

In [ ]:
with open("b4shy.github.io/index.html", "w") as f:
    f.write(random_blogpost)

In [ ]:
update_blog()

Now, let's check if our blog got updated! Note that it might take a minute or two.

One problem remains: We always overwrite the starting page!
It would be much better to create a new html file and create a link to this file on the homepage. 
Ideally, the routine would automatically assign a file name and make sure that nothing is overwritten!

In [6]:
PATH_TO_BLOG_REPO.parent

PosixPath('/Users/marci/GIT/OpenAI-API-Python-Course/04-blogpost/b4shy.github.io')

In [7]:
def create_new_blog(content):
    path_to_blog = PATH_TO_BLOG_REPO.parent
    
    path_to_content = path_to_blog/"content"
    path_to_content.mkdir(exist_ok=True, parents=True)
    
    files = len(os.listdir(path_to_content))
    new_title = f"{files+1}.html"
    path_to_new_content = path_to_content/new_title
    
    
    if not os.path.exists(path_to_new_content):
        with open(path_to_new_content, "w") as f:
            f.write(content)
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exist! Abort")

In [8]:
path_to_new_content = create_new_blog(random_string(5000))

Blog created


In [9]:
path_to_new_content

PosixPath('/Users/marci/GIT/OpenAI-API-Python-Course/04-blogpost/b4shy.github.io/content/1.html')

Now all we have to do in order to access our new blog is to add the link to index.html.
Let's manually overwrite index.html with the following code.

After that we can use beautifulsoup to automatically add the link to index.html

In [10]:
with open("b4shy.github.io/index.html") as index:
    soup = Soup(index.read())
str(soup)

'<!DOCTYPE html>\n\n<html>\n<head>\n<title>My Blog</title>\n</head>\n<body>\n<h1>My Blog</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

In [11]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls
    

In [12]:
def write_to_index(path_to_new_content):
    with open("b4shy.github.io/index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open("b4shy.github.io/index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [13]:
write_to_index(path_to_new_content)

Last but not least we can push the new blogpost to the repostory!

In [16]:
update_blog()